# Combining Multiple Excel Files with Pandas

## Introduction

One of the most common tasks for pandas and python is to automate the process to aggregate data from multiple spreadsheets and files.

This article will walk through the basic flow required to parse multiple excel files, combine some data, clean it up and analyze it.

Please refer to [Combining Data From Multiple Excel Files](https://pbpython.com/excel-file-combine.html) for the full post.

## Collecting the Data

Import pandas and numpy

In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
WIN = 'Windows'
NIX = 'Linux'
OSX = 'OSX'
platforms = {
    'linux' : NIX,
    'linux1' : NIX,
    'linux2' : NIX,
    'darwin' : OSX,
    'win32' : WIN
}

Let's take a look at the files in our input directory, using the convenient shell commands in ipython.

In [3]:
if platforms[sys.platform] == WIN:
    !dir "../data"
elif platforms[sys.platform] == NIX  or os_platform == OSX:
    !ls ../data

2018_Sales_Total_Tabs.xlsx		  sales-estimate.xlsx
2018_Sales_Total_v2.xlsx		  sales-feb-2014 (1).xlsx
2018_Sales_Total.xlsx			  sales-feb-2014.xlsx
All-Web-Site-Data-Audience-Overview.xlsx  salesfunnel.xlsx
Aussie_Wines_Plotting.csv		  sales-jan-2014(1).xlsx
cereal_data.csv				  sales-jan-2014.xlsx
customer-status (1).xlsx		  sales-mar-2014 (1).xlsx
customer-status.xlsx			  sales-mar-2014.xlsx
hospital_account_dupes.csv		  sales_transactions.xlsx
hospital_account_info.csv		  sample-address-1.xlsx
hospital_reimbursement.csv		  sample-address-2.xlsx
March-2017-forecast-article.xlsx	  sample-sales-reps.xlsx
mn-budget-detail-2014.csv		  sample-sales-tax.csv
MN_Traffic_Fatalities.csv		  sample-salesv2.csv
Online_Retail.xlsx			  sample-salesv3.xlsx
sales_cleanup.xlsx			  school_transform.csv
sales_data_types.csv			  shipping_tables.xlsx
sales-estimate.csv			  Traffic_20170306-20170519.xlsx


There are a lot of files, but we only want to look at the `sales-*-2014.xlsx` files.

In [4]:
if platforms[sys.platform] == WIN:
    !dir "../data/sales-*-2014.xlsx"
elif platforms[sys.platform] == NIX  or os_platform == OSX:
    !ls ../data/sales-*-2014.xlsx

../data/sales-feb-2014.xlsx  ../data/sales-mar-2014.xlsx
../data/sales-jan-2014.xlsx


Use the python `glob` module to easily list out the files we need

In [5]:
import glob
glob.glob("../data/sales-*-2014.xlsx")

['../data/sales-feb-2014.xlsx',
 '../data/sales-mar-2014.xlsx',
 '../data/sales-jan-2014.xlsx']

This gives us what we need, let's import each of our files and combine them into one file. Panda's `concat` and `append` can do this for us. I'm going to use `append` in this example.

The code snippet below will initialize a blank DataFrame then append all of the individual files into the `all_data` DataFrame.

In [6]:
all_data = pd.DataFrame()
for f in glob.glob("../data/sales-*-2014.xlsx"):
    df = pd.read_excel(f)
    all_data = all_data.append(df,ignore_index=True)

Now we have all the data in our all_data DataFrame. You can use describe to look at it and make sure you data looks good.

In [7]:
all_data.describe()

,account number,quantity,unit price,ext price
count,384.000000,384.000000,384.000000,384.000000
mean,478125.989583,24.372396,56.651406,1394.517344
std,220902.947401,14.373219,27.075883,1117.809743
min,141962.000000,-1.000000,10.210000,-97.160000
25%,257198.000000,12.000000,32.612500,482.745000
50%,424914.000000,23.500000,58.160000,1098.710000
75%,714466.000000,37.000000,80.965000,2132.260000
max,786968.000000,49.000000,99.730000,4590.810000


Alot of this data may not make much sense for this data set but I'm most interested in the count row to make sure the number of data elements makes sense.

In [8]:
all_data.head()

,account number,name,sku,quantity,unit price,ext price,date
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32
3,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70,2014-02-01 19:56:48
4,672390,Kuhn-Gusikowski,S1-06532,48,55.82,2679.36,2014-02-02 03:45:20


It is not critical in this example but the best practice is to convert the date column to a date time object.

In [9]:
all_data['date'] = pd.to_datetime(all_data['date'])

## Combining Data

Now that we have all of the data into one DataFrame, we can do any manipulations the DataFrame supports. In this case, the next thing we want to do is read in another file that contains the customer status by account. You can think of this as a company's customer segmentation strategy or some other mechanism for identifying their customers.

First, we read in the data.

In [ ]:
status = pd.read_excel("../data/customer-status.xlsx")
status

We want to merge this data with our concatenated data set of sales. We use panda's merge function and tell it to do a left join which is similar to Excel's vlookup function.

In [ ]:
all_data_st = pd.merge(all_data, status, how='left')
all_data_st.head()

This looks pretty good but let's look at a specific account.

In [ ]:
all_data_st[all_data_st["account number"]==737550].head()

This account number was not in our status file, so we have a bunch of NaN's. We can decide how we want to handle this situation. For this specific case, let's label all missing accounts as bronze. Use the fillna function to easily accomplish this on the status column.

In [ ]:
all_data_st['status'].fillna('bronze',inplace=True)
all_data_st.head()

Check the data just to make sure we're all good.

In [ ]:
all_data_st[all_data_st["account number"]==737550].head()

Now we have all of the data along with the status column filled in. We can do our normal data manipulations using the full suite of pandas capability.

## Using Categories

One of the relatively new functions in pandas is support for categorical data. From the pandas, documentation -

"Categoricals are a pandas data type, which correspond to categorical variables in statistics: a variable, which can take on only a limited, and usually fixed, number of possible values (categories; levels in R). Examples are gender, social class, blood types, country affiliations, observation time or ratings via Likert scales."

For our purposes, the status field is a good candidate for a category type.

You must make sure you have a recent version of pandas installed for this example to work.

In [ ]:
pd.__version__

First, we typecast it to a category using astype.

In [ ]:
all_data_st["status"] = all_data_st["status"].astype("category")

This doesn't immediately appear to change anything yet.

In [ ]:
all_data_st.head()

Buy you can see that it is a new data type.

In [ ]:
all_data_st.dtypes

Categories get more interesting when you assign order to the categories. Right now, if we call sort on the column, it will sort alphabetically. 

In [ ]:
all_data_st.sort_values(by=["status"]).head()

We use set_categories to tell it the order we want to use for this category object. In this case, we use the Olympic medal ordering.

In [ ]:
 all_data_st["status"].cat.set_categories([ "gold","silver","bronze"],inplace=True)

Now, we can sort it so that gold shows on top.

In [ ]:
all_data_st.sort_values(by=["status"]).head()

In [ ]:
all_data_st["status"].describe()

For instance, if you want to take a quick look at how your top tier customers are performaing compared to the bottom. Use groupby to give us the average of the values.

In [ ]:
all_data_st.groupby(["status"])["quantity","unit price","ext price"].mean()

Of course, you can run multiple aggregation functions on the data to get really useful information 

In [ ]:
all_data_st.groupby(["status"])["quantity","unit price","ext price"].agg([np.sum,np.mean, np.std])

So, what does this tell you? Well, the data is completely random but my first observation is that we sell more units to our bronze customers than gold. Even when you look at the total dollar value associated with bronze vs. gold, it looks backwards.

Maybe we should look at how many bronze customers we have and see what is going on.

What I plan to do is filter out the unique accounts and see how many gold, silver and bronze customers there are.

I'm purposely stringing a lot of commands together which is not necessarily best practice but does show how powerful pandas can be. Feel free to review my previous articles and play with this command yourself to understand what all these commands mean.

In [ ]:
all_data_st.drop_duplicates(subset=["account number","name"]).iloc[:,[0,1,7]].groupby(["status"])["name"].count()

Ok. This makes a little more sense. We see that we have 9 bronze customers and only 4 customers. That is probably why the volumes are so skewed towards our bronze customers.

---
*EOF*